# Setting up notebook for processing Sentinel-1 ISCE Bursts teken from the ASF Vertex website

In [ ]:
# importing needed libraries for the entire notebook in one go
%matplotlib inline
%matplotlib widget
from pathlib import Path
from dateutil.parser import parse as parse_date
from datetime import datetime
import pandas as pd
import numpy as np
from osgeo import gdal, osr
import matplotlib.pyplot as plt
from typing import List, Union
from mintpy.cli import view, tsview

# only need these two if you plan on searching andrequesting for data from within this notebook
# in my opinion, while it may take longer, it is much easier to use the ASF Vertex Website
# that said, if you are monitoring a reoccuring site and just need more imagery, searching using the notebook would be best
import asf_search as asf
import hyp3_sdk as sdk

# Set parameters for search and storage
- edit to include flightDirection and perpendicular baseline?

In [ ]:
def project_dir(project_name):
    """
    This function reads in a string that you wish to make your working directory 
    for the InSAR project, and creates a data directory to store the data from ASF
    """

    work_dir = Path.cwd() / project_name
    data_dir = work_dir / 'data'
    data_dir.mkdir(parents=True, exist_ok=True)

    return work_dir, data_dir

def get_sbas_pairs(asf_user, asf_pass, start_date, end_date, max_temp_baseline, aoi, pol): # add perpendicular baseline options - smaller baseline may lead to better interferogram formation
    """
    This function reads in a start and date for image searching,
    as well as a maximum temporal baseline between images.

    It should be noted that smaller temporal baselines tend to present
    more coherence, with a closure phase closer to 0 when multilooking
    
    asf_user = username for NASA earth data login
    asf_pass = password for NASA earth data login
    start_date = 'YYYY-MM-DD'
    start_date = 'YYYY-MM-DD'
    max_temp_baseline = integer, greater than 5 as that is the shortest S1 revisit possible
    aoi = the drawn aoi, scroll down
    polarization = asf.POLARIZATION.VV or asf.POLARIZATION.VH
    """
    
    session = asf.ASFSession()
    session.auth_with_creds(asf_user, asf_pass)

    start = parse_date(start_date + ' 00:00:00Z')
    end = parse_date(end_date + ' 00:00:00Z')
    
    search_results = asf.search(
        platform = asf.PLATFORM.SENTINEL1,
        polarization = pol,
        intersectsWith= aoi,
        start = start,
        end = end,
        processingLevel= asf.PRODUCT_TYPE.BURST,
        beamMode= asf.BEAMMODE.IW
        # direction=direction
    )
    
    baseline_results = asf.baseline_search.stack_from_product(search_results[-1])

    cols = list(baseline_results[0].properties.keys()) + ['geometry', ]
    data = [list(scene.properties.values()) + [scene.geometry, ] for scene in baseline_results]

    stack = pd.DataFrame(data, columns=cols)
    stack['startTime'] = stack.startTime.apply(parse_date)

    stack = stack.loc[(start <= stack.startTime) & (stack.startTime <= end)]

    sbas_pairs = set()

    for ref, rt in stack.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
        secondaries = stack.loc[
            (stack.sceneName != ref)
            & (stack.temporalBaseline - rt <= max_temp_baseline)
            & (stack.temporalBaseline - rt > 0)
        ]
        for sec in secondaries.sceneName:
            sbas_pairs.add((ref, sec))

    print(sbas_pairs)
    return sbas_pairs

def order_pairs(proj_name, usr, passwrd, water_mask, multilook, sbas_pairs):
    """
    This function reads in a start and date for image searching,
    as well as a maximum temporal baseline between images.

    It should be noted that smaller temporal baselines tend to present
    more coherence, with a closure phase closer to 0 when multilooking
    
    proj_name = name of your project as a string, make it specific as you may need multiple projects
    usr = NASA earthdata username
    passwrd = NASA earthdata password
    water_mask = False or True, True will mask out incoherent water bodies
    multilook = '20x4' resulting in 160m/pixel, '10x2' resulting in 80m/pixel, or '5x1' resulting in 40m/pixel are the only options
    """
    
    hyp3 = sdk.HyP3(username = usr, password = passwrd)

    jobs = sdk.Batch()

    for pair in list(sbas_pairs):
        jobs += hyp3.submit_insar_isce_burst_job(
            granule1 = pair[0],
            granule2 = pair[1],
            apply_water_mask = water_mask,
            name = proj_name,
            looks = multilook ### I need to look more into this. Can I extract the closure phase from these looks? If not must create interferograms manually so I can extract the needed info
        )
    
    jobs = hyp3.watch(jobs)

    now = datetime.now()
    start_today = datetime(now.year, now.month, now.day)

    jobs = hyp3.find_jobs(name = proj_name, start=start_today)

    insar_products = jobs.download_files(data_dir)
    insar_products = [sdk.util.extract_zipped_product(ii) for ii in insar_products]

    return insar_products

# Set working directory

In [ ]:
work_dir, data_dir = project_dir('Sentinel-1')

# Draw Area of Interest for your data search

In [ ]:
import geemap

## interactive map for you to draw a polygon to signify your aoi

## Create a map centered at a specific location
m = geemap.Map(center=[20, 0], zoom=2, basemap='ROADMAP')

## Add drawing tools
m.add_draw_control()

## Display the map
display(m)

In [ ]:
# get feature you just drew
draw_features = m.draw_features[0]

# extract the vertices pf the feature
vertices = [(coords[0], coords[1]) for i, coords in enumerate(draw_features.getInfo()['geometry']['coordinates'][0])]

# create POLYGON string to use when searching asf for imagery
aoi = "POLYGON((" + ", ".join([f'{northing} {easting}' for northing, easting in vertices[:]]) +"))"

# Find sbas isce burst pairs for insar time-series

In [ ]:
sbas_pairs = get_sbas_pairs(user name, password,'2024-01-01', '2024-03-01', 48, aoi, asf.POLARIZATION.VV)

# Download isce burst interferograms for time-series processing

In [ ]:
insar_product = order_pairs(request/project name,user name, password, True, '5x1', sbas_pairs)